In [35]:
import argparse
import py_trees
import sys
import time
#selector
"""
content = "Higher priority switching and interruption in the children of a selector.\n"
content += "In this example the higher priority child is setup to fail initially,\n"
content += "falling back to the continually running second child. On the third\n"
content += "tick, the first child succeeds and cancels the hitherto running child.\n"
"""
#sequence:
"""
"A sequence is populated with 2-tick jobs that are allowed to run through to completion.\n"
"""

def create_root():
    root = py_trees.composites.Sequence("Sequence")
    for action in ["Action 1", "Action 2", "Action 3"]:
        success_after_two = py_trees.behaviours.Count(name=action,
                                                      fail_until=0,
                                                      running_until=1,
                                                      success_until=10)
        root.add_child(success_after_two)
        
        

    root2 = py_trees.composites.Selector("Selector")
    for action in ["Action A", "Action B"]:
        success_after_two2 = py_trees.behaviours.Count(name=action,
                                                  fail_until=2,
                                                  running_until=2,
                                                  success_until=10)
        
        root2.add_child(success_after_two2)
    success_after_two3 = py_trees.behaviours.Count(name="Action C",
                                                  fail_until=0,
                                                  running_until=1,
                                                  success_until=10)
    root2.add_child(success_after_two3)
    #always_running = py_trees.behaviours.Running(name="Running")
    #root.add_children([success_after_two2, always_running])
    #root2.add_children([success_after_two2])
    root.add_children([root2])
    return root


##############################################################################
# Main
##############################################################################

def main():
    """
    Entry point for the demo script.
    """
    #args = command_line_argument_parser().parse_args()
    #print(description())
    #py_trees.logging.level = py_trees.logging.Level.DEBUG

    root = create_root()

    #py_trees.display.render_dot_tree(root)  #output tree images

    ####################
    # Execute
    ####################
    root.setup_with_descendants()
    
    status=root.update()
    print(status)
    print(type(status))
    i=1
    #for i in range(1, 6):
    while status==py_trees.common.Status.RUNNING or status==py_trees.common.Status.INVALID :
            try:
                print("\n--------- Tick {0} ---------\n".format(i))
                #print(root.Status)
                root.tick_once()

                print("\n")
                print(py_trees.display.ascii_tree(root=root, show_status=True))
                i=i+1

                time.sleep(1.0)
                status=root.update()
                print(status)
            
            except KeyboardInterrupt:
                print("interrupt")
                break
    print("\n")
    print("complete")

In [36]:
main()

Status.INVALID
<enum 'Status'>

--------- Tick 1 ---------



[-] Sequence [*]
    --> Action 1 [*] -- running
    --> Action 2 [-]
    --> Action 3 [-]
    [o] Selector [-]
        --> Action A [-]
        --> Action B [-]
        --> Action C [-]

Status.INVALID

--------- Tick 2 ---------



[-] Sequence [*]
    --> Action 1 [o]
    --> Action 2 [*] -- running
    --> Action 3 [-]
    [o] Selector [-]
        --> Action A [-]
        --> Action B [-]
        --> Action C [-]

Status.INVALID

--------- Tick 3 ---------



[-] Sequence [*]
    --> Action 1 [o]
    --> Action 2 [o]
    --> Action 3 [*] -- running
    [o] Selector [-]
        --> Action A [-]
        --> Action B [-]
        --> Action C [-]

Status.INVALID

--------- Tick 4 ---------



[-] Sequence [*]
    --> Action 1 [o]
    --> Action 2 [o]
    --> Action 3 [o]
    [o] Selector [*]
        --> Action A [x]
        --> Action B [x]
        --> Action C [*] -- running

interrupt


complete
